In [1]:
import os
from dotenv import load_dotenv
load_dotenv

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D1BB960250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D1BB960160>, model_name='llama-3.1-8b-instant')

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

message = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello how are you?")
]

result=model.invoke(message)
result

AIMessage(content='Bonjour, comment allez-vous ?', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 47, 'total_tokens': 55, 'completion_time': 0.019456151, 'completion_tokens_details': None, 'prompt_time': 0.003475694, 'prompt_tokens_details': None, 'queue_time': 0.088938132, 'total_time': 0.022931845}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b60ec17-6e43-4093-998f-546bedfd5008-0', usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55})

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ?'

In [7]:
# LCEL -> Chain the components
chain=model|parser
chain.invoke(message)

'Bonjour, comment ça va ?'

In [8]:
# Prompt template
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)

In [10]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:'), HumanMessage(content='Hello')])

In [11]:
result.to_messages()

[SystemMessage(content='Translate the following into French:'),
 HumanMessage(content='Hello')]

In [12]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour.'